In [1]:
import os, time
from multiprocessing import Process, Manager
from PIL import Image, ImageDraw, ImageFont
from google.colab import drive

drive.mount('/content/drive')


input_base = "/content/drive/MyDrive/Colab Notebooks/data_set"
output_base = "/content/output_distributed"
os.makedirs(output_base, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def process_image(input_path, output_path):
    try:
        img = Image.open(input_path).convert("RGBA")
        img_resized = img.resize((128, 128))

        watermark_text = "© Cybil Fatima"
        try:
            font = ImageFont.truetype("DejaVuSans-Bold.ttf", 12)
        except:
            font = ImageFont.load_default()

        overlay = Image.new('RGBA', img_resized.size, (255,255,255,0))
        draw = ImageDraw.Draw(overlay)
        bbox = draw.textbbox((0,0), watermark_text, font=font)
        textwidth = bbox[2] - bbox[0]
        textheight = bbox[3] - bbox[1]
        x = img_resized.width - textwidth - 6
        y = img_resized.height - textheight - 6
        draw.text((x, y), watermark_text, font=font, fill=(255,255,255,180))

        watermarked = Image.alpha_composite(img_resized, overlay).convert("RGB")
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        watermarked.save(output_path)
    except Exception as e:
        print(f"Error: {e}")


def node_worker(node_id, paths_subset, timings):
    start = time.time()
    for input_path, output_path in paths_subset:
        process_image(input_path, output_path)
    end = time.time()
    timings[node_id] = (len(paths_subset), end - start)

def get_all_paths():
    all_paths = []
    for root, _, files in os.walk(input_base):
        for f in files:
            if f.lower().endswith(('.png', '.jpg', '.jpeg')):
                rel = os.path.relpath(root, input_base)
                out_folder = os.path.join(output_base, rel)
                os.makedirs(out_folder, exist_ok=True)
                in_path = os.path.join(root, f)
                out_path = os.path.join(out_folder, f)
                all_paths.append((in_path, out_path))
    return all_paths

if __name__ == "__main__":
    all_paths = get_all_paths()
    total = len(all_paths)
    half = total // 2

    node1_paths = all_paths[:half]
    node2_paths = all_paths[half:]

    manager = Manager()
    timings = manager.dict()

    start_total = time.time()

    p1 = Process(target=node_worker, args=(1, node1_paths, timings))
    p2 = Process(target=node_worker, args=(2, node2_paths, timings))

    p1.start()
    p2.start()
    p1.join()
    p2.join()

    end_total = time.time()

    node1_imgs, node1_time = timings[1]
    node2_imgs, node2_time = timings[2]

    print(f"Node 1 processed {node1_imgs} images in {node1_time:.2f}s")
    print(f"Node 2 processed {node2_imgs} images in {node2_time:.2f}s")

    total_time = end_total - start_total
    print(f"Total distributed time: {total_time:.2f}s")

    sequential_time = 18.24
    efficiency = sequential_time / total_time
    print(f"Efficiency: {efficiency:.2f}x over sequential")

Node 1 processed 47 images in 18.41s
Node 2 processed 47 images in 31.40s
Total distributed time: 31.43s
Efficiency: 0.58x over sequential
